# Kapitola 17: Nejistota a predikce - Prezije pasazer Titanicu?

**Blok 2: Prohledavaci algoritmy a optimalizace**

---

## Co se naucite

V teto kapitole:
- Pochopite, co je **nejistota v datech** a proc je pro AI normalni
- Naucite se **cistit data** a **doplnovat chybejici hodnoty**
- Postavite **predikcni model** pomoci Random Forest
- Pochopite **pravdepodobnostni predikce** - jak jista si je AI
- Analyzujete slavny **Titanic dataset** z Kaggle

---

## Motivace: Realny svet je spinavy

V predchozich kapitolach jsme pracovali s cistymy daty. Ale v realnem svete:
- Data **chybi** (neznamy vek pasazera)
- Data jsou **nepresna** (preklepy ve jmenech)
- Data obsahuji **sum** (chybne zaznamy)

AI se musi naucit pracovat s **nejistotou**. Misto "ano/ne" rika "75% ano".

In [ ]:
# Instalace a import knihoven
!pip install seaborn scikit-learn pandas numpy matplotlib -q

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Nastaveni vizualizaci
plt.style.use('seaborn-v0_8-whitegrid')

print("Knihovny nacteny!")
print("Pripraveno pro analyzu Titanicu.")

---

## Cast 1: Nacteni a prozkoumani dat

### Titanic Dataset

Slavny dataset z Kaggle obsahuje informace o pasazerech Titanicu:
- Kdo prezil a kdo ne
- Pohlavi, vek, trida
- Rodina na palube
- Cena listku

In [ ]:
# Nacteni datasetu ze Seaborn
titanic = sns.load_dataset('titanic')

print("Titanic Dataset")
print("="*50)
print(f"Pocet zaznamu: {len(titanic)}")
print(f"Pocet sloupcu: {len(titanic.columns)}")
print("")
print("Prvnich 5 radku:")
titanic.head()

In [ ]:
# Zakladni statistiky
print("Zakladni informace o datasetu:")
print("="*50)
titanic.info()

In [ ]:
# Kolik lidi prezilo?
preziti = titanic['survived'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Graf 1: Pie chart
axes[0].pie(preziti.values, labels=['Neprezili', 'Prezili'], 
            autopct='%1.1f%%', colors=['lightcoral', 'lightgreen'],
            explode=(0.05, 0))
axes[0].set_title('Pomer preziti na Titanicu')

# Graf 2: Bar chart
preziti.plot(kind='bar', ax=axes[1], color=['lightcoral', 'lightgreen'])
axes[1].set_xticklabels(['Neprezili', 'Prezili'], rotation=0)
axes[1].set_ylabel('Pocet pasazeru')
axes[1].set_title('Pocet prezivich a neprezivich')

for i, v in enumerate(preziti.values):
    axes[1].text(i, v + 10, str(v), ha='center', fontsize=12)

plt.tight_layout()
plt.show()

print(f"\nCelkova sance na preziti: {preziti[1] / len(titanic) * 100:.1f}%")

---

## Cast 2: Analyza chybejicich dat

Realny svet je plny chybejicich dat. Podivejme se, kde nam data chybi.

In [ ]:
# Analyza chybejicich hodnot
chybejici = titanic.isnull().sum()
chybejici_procent = (chybejici / len(titanic)) * 100

# Vytvorime tabulku
chybejici_df = pd.DataFrame({
    'Sloupec': chybejici.index,
    'Chybejici': chybejici.values,
    'Procent': chybejici_procent.values
})
chybejici_df = chybejici_df[chybejici_df['Chybejici'] > 0].sort_values('Chybejici', ascending=False)

print("CHYBEJICI HODNOTY V DATASETU")
print("="*50)
print(chybejici_df.to_string(index=False))
print("="*50)

# Vizualizace
if len(chybejici_df) > 0:
    plt.figure(figsize=(10, 5))
    plt.bar(chybejici_df['Sloupec'], chybejici_df['Procent'], color='salmon')
    plt.xlabel('Sloupec')
    plt.ylabel('Chybejici hodnoty (%)')
    plt.title('Procento chybejicich hodnot v kazdem sloupci')
    plt.xticks(rotation=45)
    
    for i, (s, p) in enumerate(zip(chybejici_df['Sloupec'], chybejici_df['Procent'])):
        plt.text(i, p + 1, f'{p:.1f}%', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

---

## Cast 3: Cisteni a priprava dat

### Strategie pro chybejici hodnoty

1. **Odstraneni** - pokud chybi prilis mnoho (deck)
2. **Doplneni medianem** - pro ciselne hodnoty (age)
3. **Doplneni modem** - pro kategoricke hodnoty (embarked)

In [ ]:
# Vytvorime kopii pro praci
df = titanic.copy()

print("CISTENI DAT")
print("="*50)

# 1. Odstranime sloupce s prilis mnoho chybejicimi hodnotami
sloupce_k_odstraneni = ['deck', 'embark_town', 'alive']  # alive je duplikat survived
df = df.drop(columns=sloupce_k_odstraneni)
print(f"1. Odstraneny sloupce: {sloupce_k_odstraneni}")

# 2. Doplnime chybejici vek medianem
median_vek = df['age'].median()
df['age'] = df['age'].fillna(median_vek)
print(f"2. Chybejici vek doplnen medianem: {median_vek} let")

# 3. Doplnime chybejici misto nalodeni modem
mod_embarked = df['embarked'].mode()[0]
df['embarked'] = df['embarked'].fillna(mod_embarked)
print(f"3. Chybejici misto nalodeni doplneno modem: {mod_embarked}")

# 4. Odstranime zbyvajici radky s chybejicimi hodnotami
pred = len(df)
df = df.dropna()
po = len(df)
print(f"4. Odstraneno {pred - po} radku s chybejicimi hodnotami")

print("="*50)
print(f"Finalni pocet zaznamu: {len(df)}")
print(f"Chybejici hodnoty: {df.isnull().sum().sum()}")

In [ ]:
# Podivejme se na vycistena data
print("Vycistena data:")
df.head()

---

## Cast 4: Pruzkumova analyza (EDA)

Kdo mel vetsi sanci prezit?

In [ ]:
# Preziti podle pohlavi
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Podle pohlavi
preziti_pohlavi = df.groupby('sex')['survived'].mean() * 100
preziti_pohlavi.plot(kind='bar', ax=axes[0], color=['steelblue', 'coral'])
axes[0].set_title('Sance na preziti podle pohlavi')
axes[0].set_ylabel('Sance na preziti (%)')
axes[0].set_xticklabels(['Zena', 'Muz'], rotation=0)
axes[0].set_ylim(0, 100)
for i, v in enumerate(preziti_pohlavi.values):
    axes[0].text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=11)

# 2. Podle tridy
preziti_trida = df.groupby('class')['survived'].mean() * 100
preziti_trida.plot(kind='bar', ax=axes[1], color=['gold', 'silver', 'brown'])
axes[1].set_title('Sance na preziti podle tridy')
axes[1].set_ylabel('Sance na preziti (%)')
axes[1].set_xticklabels(['1. trida', '2. trida', '3. trida'], rotation=0)
axes[1].set_ylim(0, 100)
for i, v in enumerate(preziti_trida.values):
    axes[1].text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=11)

# 3. Podle veku
axes[2].hist([df[df['survived']==0]['age'], df[df['survived']==1]['age']], 
             label=['Neprezili', 'Prezili'], bins=20, color=['lightcoral', 'lightgreen'])
axes[2].set_title('Rozdeleni veku podle preziti')
axes[2].set_xlabel('Vek')
axes[2].set_ylabel('Pocet pasazeru')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\nKLICOVA ZJISTENI:")
print(f"- Zeny mely {preziti_pohlavi['female']:.1f}% sanci prezit (muzi jen {preziti_pohlavi['male']:.1f}%)")
print(f"- 1. trida: {preziti_trida['First']:.1f}%, 3. trida: {preziti_trida['Third']:.1f}%")
print("- 'Zeny a deti napred' bylo skutecne dodrzovano!")

---

## Cast 5: Priprava dat pro model

Model nerozumi textu jako "male" nebo "female". Musime data prevest na cisla.

In [ ]:
# Priprava features pro model
print("PRIPRAVA DAT PRO MODEL")
print("="*50)

# Vybereme relevantni sloupce
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
target = 'survived'

# Vytvorime DataFrame pro model
X = df[features].copy()
y = df[target].copy()

print(f"Features: {features}")
print(f"Target: {target}")

# Prevod kategorickych promennych na dummy promenne (one-hot encoding)
X = pd.get_dummies(X, columns=['sex', 'embarked'], drop_first=True)

print(f"\nPo one-hot encoding mame {len(X.columns)} features:")
print(list(X.columns))

X.head()

In [ ]:
# Rozdeleni na trenovaci a testovaci sadu
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Trenovaci sada: {len(X_train)} zaznamu")
print(f"Testovaci sada: {len(X_test)} zaznamu")
print(f"\nPomer: {len(X_train)/len(X)*100:.0f}% / {len(X_test)/len(X)*100:.0f}%")

---

## Cast 6: Trenovani modelu

### Random Forest Classifier

Random Forest je "les" rozhodovacich stromu:
- Kazdy strom se trenuje na nahodnem vzorku dat
- Finalni predikce je hlasovani vsech stromu
- Velmi robustni a presny model

In [ ]:
# Vytvoreni a trenovani modelu
print("TRENOVANI MODELU")
print("="*50)

model = RandomForestClassifier(
    n_estimators=100,      # Pocet stromu v lese
    max_depth=10,          # Maximalni hloubka stromu
    random_state=42        # Pro reprodukovatelnost
)

# Trenovani
model.fit(X_train, y_train)

print("Model natrenovan!")
print(f"Pocet stromu: {model.n_estimators}")
print(f"Pocet features: {model.n_features_in_}")

In [ ]:
# Evaluace modelu
y_pred = model.predict(X_test)

presnost = accuracy_score(y_test, y_pred)

print("VYSLEDKY MODELU")
print("="*50)
print(f"Presnost: {presnost*100:.2f}%")
print("")
print("Detailni report:")
print(classification_report(y_test, y_pred, target_names=['Neprezil', 'Prezil']))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Neprezil', 'Prezil'],
            yticklabels=['Neprezil', 'Prezil'])
plt.xlabel('Predikce modelu')
plt.ylabel('Skutecnost')
plt.title('Confusion Matrix - Jak casto se model myli?')
plt.show()

print("\nInterpretace:")
print(f"- Spravne predikoval nepreziti: {cm[0,0]}")
print(f"- Spravne predikoval preziti: {cm[1,1]}")
print(f"- Chybne predikoval preziti (false positive): {cm[0,1]}")
print(f"- Chybne predikoval nepreziti (false negative): {cm[1,0]}")

---

## Cast 7: Pravdepodobnostni predikce

Model nam muze rict nejen ANO/NE, ale take **jak moc si je jisty**.

In [ ]:
# Pravdepodobnostni predikce
y_proba = model.predict_proba(X_test)

# Ukazeme prvnich 10 pasazeru
print("PRAVDEPODOBNOSTNI PREDIKCE")
print("="*70)
print(f"{'Pasazer':<10} {'P(neprezil)':<15} {'P(prezil)':<15} {'Predikce':<12} {'Skutecnost'}")
print("-"*70)

for i in range(10):
    p_neprezil = y_proba[i, 0]
    p_prezil = y_proba[i, 1]
    predikce = 'Prezil' if y_pred[i] == 1 else 'Neprezil'
    skutecnost = 'Prezil' if y_test.iloc[i] == 1 else 'Neprezil'
    spravne = 'V' if predikce == skutecnost else 'X'
    
    print(f"{i+1:<10} {p_neprezil*100:>10.1f}%     {p_prezil*100:>10.1f}%     {predikce:<12} {skutecnost} {spravne}")

print("="*70)
print("V = spravna predikce, X = chybna predikce")

In [ ]:
# Rozdeleni pravdepodobnosti
plt.figure(figsize=(12, 5))

# Histogram pravdepodobnosti preziti
plt.subplot(1, 2, 1)
plt.hist(y_proba[:, 1], bins=20, color='steelblue', edgecolor='black')
plt.xlabel('Pravdepodobnost preziti')
plt.ylabel('Pocet pasazeru')
plt.title('Rozdeleni pravdepodobnosti preziti')
plt.axvline(x=0.5, color='red', linestyle='--', label='Hranice rozhodnuti')
plt.legend()

# Rozdeleni podle skutecneho preziti
plt.subplot(1, 2, 2)
prezili_idx = y_test == 1
neprezili_idx = y_test == 0

plt.hist(y_proba[prezili_idx.values, 1], bins=15, alpha=0.7, 
         label='Skutecne prezili', color='green')
plt.hist(y_proba[neprezili_idx.values, 1], bins=15, alpha=0.7, 
         label='Skutecne neprezili', color='red')
plt.xlabel('Pravdepodobnost preziti (predikce modelu)')
plt.ylabel('Pocet pasazeru')
plt.title('Jak dobre model rozlisuje?')
plt.axvline(x=0.5, color='black', linestyle='--')
plt.legend()

plt.tight_layout()
plt.show()

---

## Cast 8: Dulezitost features

Co nejvice ovlivnuje sanci na preziti?

In [ ]:
# Feature importance
dulezitost = pd.DataFrame({
    'Feature': X.columns,
    'Dulezitost': model.feature_importances_
}).sort_values('Dulezitost', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(dulezitost['Feature'], dulezitost['Dulezitost'], color='steelblue')
plt.xlabel('Dulezitost')
plt.ylabel('Feature')
plt.title('Dulezitost features pro predikci preziti')
plt.gca().invert_yaxis()

for i, v in enumerate(dulezitost['Dulezitost']):
    plt.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nNEJDULEZITEJSI FAKTORY:")
for i, (f, d) in enumerate(zip(dulezitost['Feature'][:5], dulezitost['Dulezitost'][:5])):
    print(f"{i+1}. {f}: {d*100:.1f}%")

---

## Cast 9: Predikce pro noveho pasazera

Pojdme predikovat sanci na preziti pro fiktivniho pasazera!

In [ ]:
def predikuj_preziti(pclass, sex, age, sibsp, parch, fare, embarked):
    """
    Predikuje sanci na preziti pro noveho pasazera.
    
    Args:
        pclass: Trida (1, 2, 3)
        sex: Pohlavi ('male', 'female')
        age: Vek
        sibsp: Pocet sourozencu/partneru na palube
        parch: Pocet rodicu/deti na palube
        fare: Cena listku
        embarked: Misto nalodeni ('S', 'C', 'Q')
    """
    # Vytvorime DataFrame pro noveho pasazera
    pasazer = pd.DataFrame({
        'pclass': [pclass],
        'sex': [sex],
        'age': [age],
        'sibsp': [sibsp],
        'parch': [parch],
        'fare': [fare],
        'embarked': [embarked]
    })
    
    # Prevedeme na format pro model
    pasazer_enc = pd.get_dummies(pasazer, columns=['sex', 'embarked'], drop_first=True)
    
    # Zajistime, ze mame vsechny sloupce
    for col in X.columns:
        if col not in pasazer_enc.columns:
            pasazer_enc[col] = 0
    
    pasazer_enc = pasazer_enc[X.columns]  # Spravne poradi sloupcu
    
    # Predikce
    predikce = model.predict(pasazer_enc)[0]
    proba = model.predict_proba(pasazer_enc)[0]
    
    return predikce, proba[1]


# Priklad 1: Zena v 1. tride
predikce, sance = predikuj_preziti(1, 'female', 25, 0, 0, 100, 'C')
print(f"Zena, 25 let, 1. trida: {sance*100:.1f}% sance prezit")

# Priklad 2: Muz v 3. tride
predikce, sance = predikuj_preziti(3, 'male', 30, 0, 0, 10, 'S')
print(f"Muz, 30 let, 3. trida: {sance*100:.1f}% sance prezit")

# Priklad 3: Dite v 2. tride
predikce, sance = predikuj_preziti(2, 'male', 8, 1, 2, 30, 'Q')
print(f"Chlapec, 8 let, 2. trida s rodinou: {sance*100:.1f}% sance prezit")

---

## Cviceni k procviceni

### Cviceni 1: Jina imputace
Zkuste doplnit chybejici vek prumerem misto medianu. Jak se zmeni presnost?

### Cviceni 2: Chytra imputace
Doplnte vek medianem podle tridy a pohlavi (zena v 1. tride ma jiny median nez muz v 3. tride).

### Cviceni 3: Vice features
Pridejte feature "rodina" = sibsp + parch. Zlepsi se model?

### Cviceni 4: Jiny model
Zkuste pouzit LogisticRegression misto RandomForest. Porovnejte presnost.

In [ ]:
# Prostor pro vase reseni

# Cviceni 1: Zkuste prumer misto medianu
# prumer_vek = titanic['age'].mean()
# ...

---

## Shrnuti kapitoly

### Co jsme se naucili:

1. **Analyza chybejicich dat** - jak je najit a vizualizovat
2. **Cisteni dat** - odstraneni, imputace medianem/modem
3. **Pruzkumova analyza** - kdo mel vetsi sanci prezit
4. **Priprava dat** - one-hot encoding, train/test split
5. **Random Forest** - trenovani a evaluace modelu
6. **Pravdepodobnostni predikce** - jak jista si je AI
7. **Feature importance** - co nejvice ovlivnuje vysledek

### Klicove techniky:

| Technika | Pouziti |
|----------|--------|
| `fillna(median)` | Doplneni chybejicich cisel |
| `fillna(mode)` | Doplneni chybejicich kategorii |
| `get_dummies()` | One-hot encoding |
| `train_test_split()` | Rozdeleni dat |
| `predict_proba()` | Pravdepodobnostni predikce |

### Dalsi kroky:

V dalsi kapitole se podivame na **Naivni Bayesuv klasifikator** - jak
pouzit pravdepodobnost k klasifikaci textu (napriklad rozpoznani spamu)!

---

*Kapitola 17 - Nejistota a predikce*